In [1]:
!pip install pandas
!pip install opencv-python
!apt-get install -y libgl1
!pip install kaggle
!pip install scipy


[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libgl1 is already the newest version (1.4.0-1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import h5py
import shutil
import cv2
import random
import uuid
import numpy as np

2024-08-25 01:44:13.395712: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-25 01:44:13.486503: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-08-25 01:44:13.509059: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-08-25 01:44:13.621627: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
tf.config.list_physical_devices()

I0000 00:00:1724550257.555002   91093 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724550257.643526   91093 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724550257.643725   91093 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [5]:
image_gen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range = 0.1,
    shear_range=0.5,
    zoom_range=0.3,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest',
    validation_split=0.3
)

In [6]:

train_image_gen = image_gen.flow_from_directory(os.path.join("new_training_data"),
                                               target_size=(130,130),
                                                color_mode='rgb',
                                               batch_size=16,
                                               class_mode='binary',subset="training")
val_image_gen = image_gen.flow_from_directory(os.path.join("new_training_data"),target_size=(130,130),color_mode='rgb',batch_size=16,class_mode='binary',subset='validation')

Found 874906 images belonging to 2 classes.
Found 374959 images belonging to 2 classes.


In [7]:
model = tf.keras.models.load_model("./isic-model-8.keras")

I0000 00:00:1724550300.236044   91093 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724550300.236243   91093 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724550300.236276   91093 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724550300.503975   91093 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1724550300.504119   91093 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-25

In [8]:
model.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 130, 130, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential (Sequential)         │ (None, 130, 130, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 2048)           │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_1 (Sequential)       │ (None, 1)              │     1,188,726 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 62,269,924 (237.54 MB)

 Trainable params: 18,746,742 (71.51 MB)

 Non-trainable params: 6,029,696 (23.00 MB)

 Optimizer params: 37,493,486 (143.03 MB)

In [9]:

model.compile(optimizer=tf.keras.optimizers.Adam(),loss=tf.keras.losses.BinaryFocalCrossentropy(
    apply_class_balancing=True,
    alpha=0.75,
    gamma=2.0,
    from_logits=False,
    label_smoothing=0.0,
    axis=-1,
    reduction='sum_over_batch_size',
    name='binary_focal_crossentropy'
),metrics=[tf.keras.metrics.BinaryAccuracy(),tf.keras.metrics.Recall(),tf.keras.metrics.AUC(),tf.keras.metrics.FalsePositives(),tf.keras.metrics.FalseNegatives()])

In [10]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',patience=1)
model.fit(train_image_gen,epochs=4, callbacks=[early_stop],validation_data=val_image_gen,batch_size=16)

Epoch 1/4


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
2024-08-25 01:45:30.338883: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8906
W0000 00:00:1724550330.542779   91253 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724550330.623429   91253 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724550330.637647   91253 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724550330.653367   91253 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724550330.667484  

    1/54682 ━━━━━━━━━━━━━━━━━━━━ 473:33:12 31s/step - auc: 0.8594 - binary_accuracy: 0.7500 - false_negatives: 2.0000 - false_positives: 2.0000 - loss: 0.0880 - recall: 0.7500

W0000 00:00:1724550340.424307   91255 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724550340.428328   91255 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724550340.432500   91255 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724550340.436406   91255 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724550340.440535   91255 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724550340.444709   91255 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724550340.448713   91255 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724550340.453455   91255 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724550340.457206   91255 gp

50516/54682 ━━━━━━━━━━━━━━━━━━━━ 18:34 268ms/step - auc: 0.9250 - binary_accuracy: 0.8252 - false_negatives: 5662.2505 - false_positives: 64077.7344 - loss: 0.0408 - recall: 0.9716

...

W0000 00:00:1724563866.172632   91251 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724563866.176640   91251 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724563866.181187   91251 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724563866.186527   91251 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724563866.191089   91251 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724563866.196205   91251 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724563866.201340   91251 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced


54682/54682 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step - auc: 0.9255 - binary_accuracy: 0.8255 - false_negatives: 6129.9810 - false_positives: 69216.9922 - loss: 0.0406 - recall: 0.9717

W0000 00:00:1724564969.759820   91253 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724564969.783305   91253 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724564969.798859   91253 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724564969.816764   91253 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724564969.833494   91253 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724564969.878720   91253 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724564969.895957   91253 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724564969.934732   91253 gpu_timer.cc:114] Skipping the delay kernel, measurement accuracy will be reduced
W0000 00:00:1724564969.950203   91253 gp

54682/54682 ━━━━━━━━━━━━━━━━━━━━ 18655s 341ms/step - auc: 0.9255 - binary_accuracy: 0.8255 - false_negatives: 6130.0942 - false_positives: 69218.2344 - loss: 0.0406 - recall: 0.9717 - val_auc: 0.9118 - val_binary_accuracy: 0.7769 - val_false_negatives: 6070.0000 - val_false_positives: 77572.0000 - val_loss: 0.0455 - val_recall: 0.9677
Epoch 2/4
54682/54682 ━━━━━━━━━━━━━━━━━━━━ 18299s 335ms/step - auc: 0.9379 - binary_accuracy: 0.8410 - false_negatives: 6255.6724 - false_positives: 63208.8867 - loss: 0.0358 - recall: 0.9709 - val_auc: 0.9174 - val_binary_accuracy: 0.7764 - val_false_negatives: 4155.0000 - val_false_positives: 79701.0000 - val_loss: 0.0387 - val_recall: 0.9779
Epoch 3/4
21245/54682 ━━━━━━━━━━━━━━━━━━━━ 2:27:06 264ms/step - auc: 0.9366 - binary_accuracy: 0.8336 - false_negatives: 2405.6946 - false_positives: 25681.4668 - loss: 0.0371 - recall: 0.9729

KeyboardInterrupt: 

In [11]:
model.save('isic-model-20.keras')